# Refit rheology and generate values in ASPECT

Here I summarized what I learned from the other jupyter notebook refit_HK03.ipynb

In the previous notebook, I refit the prefactor A after both E and V are varied.

In this notebook, I took a different approach and condense the modification in $\dot{\epsilon_{correction}}$.
This variable is computed from a original flow law with a experimental referential state.
It is then carried into viscosity calculation to be a modification of $\dot{\epsilon_{correction}}^{1/n}$ 


In [ ]:
import json
import os
import sys
import math
import argparse
import numpy as np
from scipy.special import erf
from scipy.interpolate import interp1d
from mpl_toolkits.mplot3d import Axes3D
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm
from matplotlib import patches as mpatches 

plt.style.use('publication_lhy11009')

R = 8.314

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))
import shilofue.PlotDepthAverage as PDAver
import shilofue.FlowLaws as FlowLaws
import shilofue.ParsePrm as ParsePrm
import shilofue.ThermalModel as TModel
import shilofue.Rheology as Rheology

# import utilities in subdirectiory
sys.path.append(os.path.join(ASPECT_LAB_DIR, 'utilities', "python_scripts"))
import Utilities


#### First, load the rheology
By default, I use the flow law in Warren, Hansen 2023's table.
The difference from the Hirth, Kohlstedt 2003 flow law is a change of the prefactor by $3.5^r$.
I didnt' start with the activation volume of 1.9e-6 in the diffusion creep by Siber, Korato 2022.
First use the value from Hirth, Kohlstedt 2003 of 4e-6 and then modify the value on top of it.

In [ ]:
# read the parameters from HK 03.
# water is in the water fugacity
# rheology = "HK03"
rheology = "WarrenHansen23"

rheology_prm_dict = Rheology.RHEOLOGY_PRM()
diffusion_creep_ori = getattr(rheology_prm_dict, rheology + "_diff")
dislocation_creep_ori = getattr(rheology_prm_dict, rheology + "_disl")
rheology_dict = {'diffusion': diffusion_creep_ori, 'dislocation': dislocation_creep_ori}

print("diffusion_creep: ", diffusion_creep_ori)
print("dislocation_creep: ", dislocation_creep_ori)

#### Next: assign a modification

Next, I assign a modification to the original flow law.
The modification will be added to the original variables.

The strain_rate_correction term is then calculated from the reference state.

A Coh value is also assigned to adjust the rheology.
Note this factor is not in the ASPECT's rheology module.

In [ ]:
# 1.9e-6
diff_correction = {'A': 1.0, 'p': 0.0, 'r': 0.0, 'n': 0.0, 'E': -75e3, 'V': -2.1e-6}
disl_correction = {'A': 1.0, 'p': 0.0, 'r': 0.0, 'n': 0.0, 'E': 0.0, 'V': 0.0}
Coh = 300.0

# fit new rheology
stress_ref = 50.0 # MPa
# strain_rate_ref = 1e-5
P_ref = 100.0e6 # Pa
T_ref = 1250.0 + 273.15 # K
Coh_ref = 1000.0 # H / 10^6 Si
d_ref = 15.0 # mu m

strain_rate_diff_correction = Rheology.CreepStrainRate(diff_correction, stress_ref, P_ref, T_ref, d_ref, Coh_ref)
strain_rate_disl_correction = Rheology.CreepStrainRate(disl_correction, stress_ref, P_ref, T_ref, d_ref, Coh_ref)

# screen output
print("strain_rate_diff_correction:", strain_rate_diff_correction) 
print("strain_rate_disl_correction:", strain_rate_disl_correction) 

# apply the correction
diffusion_creep = diffusion_creep_ori.copy()
dislocation_creep = dislocation_creep_ori.copy()

diffusion_creep['E'] += diff_correction['E']
diffusion_creep['V'] += diff_correction['V']
dislocation_creep['E'] += disl_correction['E']
dislocation_creep['V'] += disl_correction['V']
diffusion_creep['A'] /= strain_rate_diff_correction
dislocation_creep['A'] /= strain_rate_disl_correction

# screen output
print("diffusion_creep: ", diffusion_creep)
print("dislocation_creep: ", dislocation_creep)

#### Last: Lower mantle rheology with a jump, plots, profiles

Then I call a function "MantleRheology" to derive the whole mantle rheology profile.
The mantle profile viscosity profile is plotted and saved.
The aspect importable rheology is saved in a dict and then returned.

In [ ]:
Operator = Rheology.RHEOLOGY_OPR()

source_dir = os.path.join(ASPECT_LAB_DIR, 'tests', 'integration', 'fixtures', 'test_rheology')
da_file = os.path.join(source_dir, "depth_average.txt")
Operator.ReadProfile(da_file)

fig_path = os.path.join(ASPECT_LAB_DIR, "results", "%s_coh%.1f_dEdiff%.2e_dVdiff%.2e_dEdisl%.2e_dVdisl%.2e_1"\
                                                 % (rheology, Coh, diff_correction['E'], diff_correction['V'], disl_correction['E'], disl_correction['V']))\
                                                  + ".png"

constrained_rheology_aspect, _ = Operator.MantleRheology(assign_rheology=True, diffusion_creep=diffusion_creep, dislocation_creep=dislocation_creep, save_profile=1,\
        use_effective_strain_rate=True, save_json=False, fig_path=fig_path, Coh=Coh)


print("constrained_rheology_aspect (type):", type(constrained_rheology_aspect))
print("constrained_rheology_aspect:", constrained_rheology_aspect)

#### Additional step: check consistency with the reference state

